# Object Tracking

Objectives


1. Tracking single object using Opencv based functions
2. Tracking multiple objects using deep learning technique


In [26]:
!pip install imageio-ffmpeg
!pip install opencv-contrib-python==4.9.0.80
# !pip install uninstall opencv-python
# !pip install opencv-contrib-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.3/68.3 MB 9.4 MB/s eta 0:00:00
  Attempting uninstall: opencv-contrib-python
    Found existing installation: opencv-contrib-python 4.10.0.84
    Uninstalling opencv-contrib-python-4.10.0.84:
      Successfully uninstalled opencv-contrib-python-4.10.0.84


In [1]:
from IPython.display import clear_output
import cv2
import sys
%matplotlib inline
from matplotlib import pyplot as plt
plt.rcParams["figure.figsize"] = (16, 10) # (w, h)
import numpy as np
import time
import math
from google.colab.patches import cv2_imshow
import imutils
!pip install tf-nightly-gpu-2.0-preview

# For running inference on the TF-Hub module.
import tensorflow as tf
import tensorflow_hub as hub

# For downloading the image.
import matplotlib.pyplot as plt
import tempfile
from six.moves.urllib.request import urlopen
from six import BytesIO

# For drawing onto the image.
import numpy as np
from PIL import Image
clear_output()

In [2]:
!wget "https://drive.google.com/uc?export=view&id=1skt6SmYqjQ_jWh4jIkpe3njILEIaS72-" -O vid.mp4
!wget "https://drive.google.com/uc?id=1Ckgprba7Yd6a-nQQi9AfXvSEGsEZXpcz&authuser=0&export=download" -O cars.mp4

--2024-11-26 08:47:20--  https://drive.google.com/uc?export=view&id=1skt6SmYqjQ_jWh4jIkpe3njILEIaS72-
Resolving drive.google.com (drive.google.com)... 172.217.203.139, 172.217.203.102, 172.217.203.138, ...
Connecting to drive.google.com (drive.google.com)|172.217.203.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1skt6SmYqjQ_jWh4jIkpe3njILEIaS72-&export=view [following]
--2024-11-26 08:47:21--  https://drive.usercontent.google.com/download?id=1skt6SmYqjQ_jWh4jIkpe3njILEIaS72-&export=view
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 74.125.139.132, 2607:f8b0:400c:c05::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|74.125.139.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 489460 (478K) [video/mp4]
Saving to: ‘vid.mp4’

vid.mp4             100%[===================>] 477.99K  --.-KB/s    in 0.006s  

2024-11-26 08:

## Helper functions

In [3]:
def show_in_row(list_of_images, titles = None, disable_ticks = False):
  count = len(list_of_images)
  for idx in range(count):
    subplot = plt.subplot(1, count, idx+1)
    if titles is not None:
      subplot.set_title(titles[idx])

    img = list_of_images[idx]
    cmap = 'gray' if (len(img.shape) == 2 or img.shape[2] == 1) else None
    subplot.imshow(img, cmap=cmap)
    if disable_ticks:
      plt.xticks([]), plt.yticks([])
  plt.show()


Additional functions

In [4]:
def detect_red(frame):
  """
  Detects the largest red object in the frame and returns its bounding box.
  """
  state = []
  track_frame = cv2.inRange(frame.copy(), (0, 0, 140), (60, 60, 255))
  track_frame = cv2.dilate(track_frame, np.ones((39,39), np.uint8))
  track_frame = cv2.erode(track_frame, np.ones((29,29), np.uint8))

  contour = cv2.findContours(track_frame, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  contour = imutils.grab_contours(contour)
  if len(contour)>0:
    c = max(contour, key=cv2.contourArea)
    (cent_x, cent_y), radius = cv2.minEnclosingCircle(c)
    cent_x, cent_y, radius = int(cent_x),int(cent_y), int(radius)
    state.append((cent_x-radius, cent_y-radius, 2*radius, 2*radius))
    if (4*radius*radius)<1000:
      state = []
  return state


def create_tracker(tracker_type='KCF'):
  if tracker_type == 'BOOSTING':
      tracker = cv2.TrackerBoosting_create()
  elif tracker_type == 'MIL':
      tracker = cv2.TrackerMIL_create()
  elif tracker_type == 'KCF':
      tracker = cv2.TrackerKCF_create()
  elif tracker_type == 'TLD':
      tracker = cv2.TrackerTLD_create()
  elif tracker_type == 'MEDIANFLOW':
      tracker = cv2.TrackerMedianFlow_create()
  elif tracker_type == 'GOTURN':
      tracker = cv2.TrackerGOTURN_create()
  elif tracker_type == 'MOSSE':
      tracker = cv2.TrackerMOSSE_create()
  elif tracker_type == "CSRT":
      tracker = cv2.TrackerCSRT_create()
  else:
      print("No valid tracker type given! Using type 'KCF'.")
      tracker = cv2.TrackerKCF_create()

  return tracker

def create_multi_tracker(tracker_type='KCF'):
  if tracker_type == 'BOOSTING':
      tracker = cv2.legacy.TrackerBoosting_create()
  elif tracker_type == 'MIL':
      tracker = cv2.legacy.TrackerMIL_create()
  elif tracker_type == 'KCF':
      tracker = cv2.legacy.TrackerKCF_create()
  elif tracker_type == 'TLD':
      tracker = cv2.legacy.TrackerTLD_create()
  elif tracker_type == 'MEDIANFLOW':
      tracker = cv2.legacy.TrackerMedianFlow_create()
  elif tracker_type == 'GOTURN':
      tracker = cv2.legacy.TrackerGOTURN_create()
  elif tracker_type == 'MOSSE':
      tracker = cv2.legacy.TrackerMOSSE_create()
  elif tracker_type == "CSRT":
      tracker = cv2.legacy.TrackerCSRT_create()
  else:
      print("No valid tracker type given! Using type 'KCF'.")
      tracker = cv2.TrackerKCF_create()

  return tracker

## Detecting and Tracking of a Single Object

In [5]:
def track(video, frame, bbox, output_writer, num_frames=5):
  """
  Tracks given object for num_frames frames, draws a bounding box around the object
  and writes those frames to an output.
  """
 # Create and initialize a tracker using the function from above
  # See documentation for cv::Tracker to see how to initialize it
  # <your code here>
  tracker = create_tracker()
  ok = tracker.init(frame, bbox)

  for i in range(num_frames):
    ok, frame = video.read()
    if not ok:
        print("[function 'track'] Error reading frame from video!")
        break

   # Update the tracker with the current frame
    # If the tracking was successful, draw a rectangle/bounding box around the object
    # If not successful, draw text on the frame indicating a tracking error
    # < your code here>
    ok, bbox = tracker.update(frame)
    if ok:
      p1 = (int(bbox[0]), int(bbox[1]))
      p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
      cv2.rectangle(frame, p1,p2, (255,200,0), 2, 1)
    else:
      cv2.putText(frame, "Tracking failure", (100,80), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,0,255), 2)

    # Display the frame (in the output below or into a video file)
    # show_in_row([frame])
    output_writer.write(frame)



def detect_object_and_track(path, output_writer):
  video = cv2.VideoCapture(path)
  ok, frame = video.read()

  try:
    while(True):
      ret, frame = video.read()
      if not ret:
        print("[function 'detect_object_and_track'] Error reading frame from video!")
        video.release()
        break

      # Detect a red object. If found, draw a bounding box around the object,
      # display or write the frame (image) with the bounding box to a video
      # and track the object for the next num_frames frames.
      # If it wasn't found, just display or write the unchanged image.
      # < your code here >
      state = detect_red(frame)
      if len(state) > 0:
        bbox = state[0]
        p1 = (int(bbox[0]), int(bbox[1]))
        p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
        cv2.rectangle(frame, p1,p2, (255,0,0), 2, 1)
        output_writer.write(frame)
      else:
        output_writer.write(frame)
  except KeyboardInterrupt:
    video.release()

out = cv2.VideoWriter('output_single.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 30, (640, 360))
detect_object_and_track('vid.mp4', out)
out.release()

[function 'detect_object_and_track'] Error reading frame from video!


In [6]:

import imageio
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML

def display_video(video):
    fig = plt.figure(figsize=(3,3))  #Display size specification

    mov = []
    for i in range(len(video)):  #Append videos one by one to mov
        img = plt.imshow(video[i], animated=True)
        plt.axis('off')
        mov.append([img])

    #Animation creation
    anime = animation.ArtistAnimation(fig, mov, interval=50, repeat_delay=1000)

    plt.close()
    return anime

video = imageio.mimread('output_single.avi')  #Loading video
video = [resize(frame, (256, 256))[..., :3] for frame in video]    #Size adjustment (if necessary)
HTML(display_video(video).to_html5_video())  #Inline video display in HTML5


## Detect (via SSD) and Track Multiple Objects

In [7]:
class CarDetector():
  def __init__(self):
    module_handle = "https://tfhub.dev/google/openimages_v4/ssd/mobilenet_v2/1"
    self.detector = hub.load(module_handle).signatures['default']

  def filter_boxes(self, image, boxes, class_names, scores, max_boxes=10, min_score=0.6, class_of_interest='Car'):
    im_height, im_width = image.shape[:2]
    state=[]

    for i in range(min(boxes.shape[0], max_boxes)):
      if scores[i] >= min_score and class_names[i].decode("ascii")==class_of_interest:
        [ymin, xmin, ymax, xmax] = boxes[i]
        (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
                                  ymin * im_height, ymax * im_height)
        left, right, top, bottom = min(left, right), max(left, right), min(top, bottom), max(top, bottom)
        left, right, top, bottom  = int(left), int(right), int(top), int(bottom)
        state.append((left, top, right-left, bottom-top))
    return state

  def detect(self, img):
    converted_img  = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]
    result = self.detector(converted_img)

    result = {key:value.numpy() for key,value in result.items()}
    bbox = self.filter_boxes(img, result["detection_boxes"],
        result["detection_class_entities"], result["detection_scores"])
    return bbox


def track_multiple(video, frame, bboxes, output_writer, num_frames=5):
  """
  Tracks object for num_frames frames
  """
# Create a MultiTracker and for each bounding box, add a tracker to it
  # use the create_multi_tracker() function to add tracker
  # < your code here >

  multi_tracker = cv2.legacy.MultiTracker_create()
  tracker = create_multi_tracker()
  for bbox in bboxes:
    multi_tracker.add(tracker, frame, bbox)


  for i in range(num_frames):
    ok, frame = video.read()
    if not ok:
        break
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)


    # Feed the tracker with a new image and draw a bounding box for each
    # tracked object onto the frame
    # < your code here >
    ok, bboxes = multi_tracker.update(frame)
    for bbox in bboxes:
        p1 = (int(bbox[0]), int(bbox[1]))
        p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
        cv2.rectangle(frame, p1,p2, (255,0,0), 2, 1)

    # show_in_row([frame])
    output_writer.write(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))



def detect_object_and_track(path, output_writer, num_frames=300, start_frame=0):
  video = cv2.VideoCapture(path)
  video.set(cv2.CAP_PROP_POS_FRAMES, start_frame) # start video at ~37 sec
  ok, frame = video.read()

  try:
   while video.get(cv2.CAP_PROP_POS_FRAMES) < start_frame + num_frames:
      ret, frame = video.read()
      if not ret:
        video.release()
        break

      frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
      state = car_detect.detect(frame)
       # If we found objects, draw a bounding box around each detected object,
      # display or write the resulting image and then track all found objects.
      # If nothing was found, display or draw the unchanged image.
      # REMEMBER TO CONVERT BACK TO BGR if you are writing to a video file!
      # < your code here >
      if state is not None:
        frame_draw = frame.copy()
        for bbox in state:
          p1 = (int(bbox[0]), int(bbox[1]))
          p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
          cv2.rectangle(frame, p1,p2, (255,0,0), 2, 1)
        output_writer.write(cv2.cvtColor(frame_draw, cv2.COLOR_RGB2BGR))
        track_multiple(video,frame, state, out)
      else:
        pass
        output_writer.write(cv2.cvtColor(frame_draw, cv2.COLOR_RGB2BGR))
  except KeyboardInterrupt:
    video.release()

out = cv2.VideoWriter('output_multiple.avi', cv2.VideoWriter_fourcc('M','J','P','G'), 25, (640, 360))
car_detect = CarDetector()
detect_object_and_track("cars.mp4", out, 750, 4950)
out.release()

In [8]:
video_multiple = imageio.mimread('output_multiple.avi', memtest=False)  #Loading video
video_multiple = [resize(frame, (256, 256))[..., :3] for frame in video_multiple]    #Size adjustment (if necessary)
HTML(display_video(video_multiple).to_html5_video())  #Inline video display in HTML5